## Load packages & data

### Packages

In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

import time
import Categorical_similarity_measures as sim
import kmedoids

from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist, jaccard

from sklearn import preprocessing

In [3]:
# função frequency report
def frequency_report(data,parameter, n):
    most_frequent = data[parameter].value_counts()
    temp = most_frequent.head(n)
    
    titulos = list(temp.index)
    counts = list(temp)
    percent = list(round(temp/sum(most_frequent)*100,2))
    zipped = list(zip(titulos,counts,percent))
    
    df_ocupation = pd.DataFrame(data = zipped, columns= ["Titulo","Count","Porcentagem (%)"])
    
    return(df_ocupation)

### Data

In [ ]:
path = "~/Documents/jupyter-workplace/data-mining-assignment/reformed_all_data.csv"
data = pd.read_csv(path, index_col=0)

path_cbo = "~/Documents/jupyter-workplace/data-mining-assignment/CBO2002.csv"
cbo = pd.read_csv(path_cbo, index_col=0, encoding = 'unicode_escape', engine ='python')

In [ ]:
data.head(n = 10)

## Preprocessing

In [ ]:
# selecionando brancas, pardas e pretas
data_cluster = data[((data['RACACOR'] == 'Parda') | (data['RACACOR'] == 'Preta') | (data['RACACOR'] == 'Branca'))]

data_cluster = data_cluster.reset_index()

data_cluster = data_cluster.drop("index", axis=1)

In [ ]:
# substituir preta e pardar por preta/parda
data_cluster['RACACOR'] = data_cluster['RACACOR'].map({'Branca':'Branca','Preta':'Preta/Parda',
                                                       'Parda':'Preta/Parda'})
raca_dtype = pd.CategoricalDtype(categories=['Branca', 'Preta/Parda'], 
                                   ordered=False)
data_cluster['RACACOR'].astype(raca_dtype)

In [ ]:
# only relevant ocupations
table1 = frequency_report(data = data, parameter = 'CODOCUPMAE',n = 10)
table2 = frequency_report(data = data_branca[data_branca['ESCMAE'] == '12 e mais'],
                          parameter = 'CODOCUPMAE',n = 10)
table3 = frequency_report(data = data_pp[data_pp['ESCMAE'] != '12 e mais'],
                          parameter = 'CODOCUPMAE',n = 10)

table = list(table1["Titulo"])+list(table2["Titulo"])+list(table3["Titulo"])

top_ocupations = list(set(table))

data_cluster = data_cluster[data_cluster['CODOCUPMAE'].isin(top_ocupations)]
data_cluster.shape

In [ ]:
# "Professor da educação de jovens e adultos..." == "Professor"
data_cluster['CODOCUPMAE'] = data_cluster['CODOCUPMAE'].replace(['Professor da  educação de jovens e adultos do ensino fundamental (primeira a quarta série)'],
                                                                 'Professor')
# "Empregado doméstico diarista" == "Empregado doméstico nos serviços gerais"
data_cluster['CODOCUPMAE'] = data_cluster['CODOCUPMAE'].replace(['Empregado  doméstico  nos serviços gerais','Empregado doméstico diarista'],
                                                                'Empregado doméstico')


In [ ]:
frequency_report(data_cluster, parameter='CODOCUPMAE', n=20)

In [ ]:
# high relative remuneration
high = ['Psicólogo clínico','Enfermeiro','Advogado']
low = ['Operador de caixa','Empregado doméstico','Vendedor de comércio varejista']

selected_ocupations = high+low
data_cluster = data_cluster[data_cluster['CODOCUPMAE'].isin(selected_ocupations)]

In [ ]:
data_cluster.shape

### Sampling & Distance Matrix

Sampling while balacing ocupations...

In [ ]:
sample1 = data_cluster[data_cluster['CODOCUPMAE'] == 'Vendedor de comércio varejista'].sample(n = 500) #Vendedor de comércio varejista
sample2 = data_cluster[data_cluster['CODOCUPMAE'] == 'Operador de caixa'].sample(n = 500) #Operador de caixa
sample3 = data_cluster[data_cluster['CODOCUPMAE'] == 'Empregado doméstico'].sample(n = 500) #Empregado doméstico
sample4 = data_cluster[data_cluster['CODOCUPMAE'] == 'Advogado'].sample(n = 500) #Advogado
sample5 = data_cluster[data_cluster['CODOCUPMAE'] == 'Enfermeiro'].sample(n = 500) #Enfermeiro
sample6 = data_cluster[data_cluster['CODOCUPMAE'] == 'Psicólogo clínico'].sample(n = 500) #Psicólogo clínico

frames = [sample1,sample2,sample3,sample4,sample5,sample6]
samp = pd.concat(frames)

samp = samp.reset_index()

samp['RACACOR'].value_counts()

In [ ]:
samp.shape

In [4]:
# save sample
samp.to_csv("final_sample3000.csv")

#### Jaccard distance

In [5]:
# perfil materno
perfil_materno = ['ESCMAE','RACACOR','CODOCUPMAE']

In [6]:
# codificar elementos antes de rodar o jaccard
ocupationenc = preprocessing.LabelEncoder()
ocupationenc.fit(samp['CODOCUPMAE'])
enc_ocupation_mapping = dict(zip(ocupationenc.classes_, ocupationenc.transform(ocupationenc.classes_)))
samp['CODOCUPMAE'] = ocupationenc.transform(samp['CODOCUPMAE'])

racialenc = preprocessing.LabelEncoder()
racialenc.fit(samp['RACACOR'])
enc_racial_mapping = dict(zip(racialenc.classes_, racialenc.transform(racialenc.classes_)))
samp['RACACOR'] = racialenc.transform(samp['RACACOR'])

escolenc = preprocessing.LabelEncoder()
escolenc.fit(samp['ESCMAE'])
enc_escolaridade_mapping = dict(zip(escolenc.classes_, escolenc.transform(escolenc.classes_)))
samp['ESCMAE'] = escolenc.transform(samp['ESCMAE'])


In [7]:
# Jaccard
start = time.time() # start timer
distmat_jaccard = pdist(samp[perfil_materno], 'jaccard')
distmat_jaccard = pd.DataFrame(squareform(distmat_jaccard), index=samp.index,
                               columns= samp.index)
end = time.time() # end timer

print("time elapsed: ", (end - start)/60, " minutos")

time elapsed:  0.0019835670789082843  minutos


In [37]:
# save distance matrix
distmat_jaccard.to_csv("jaccard-distance-matrix.csv")

In [26]:
# decodificar elementos fatorizados

samp['CODOCUPMAE'] = list(ocupationenc.inverse_transform(samp['CODOCUPMAE']))
samp['RACACOR'] = list(racialenc.inverse_transform(samp['RACACOR']))
samp['ESCMAE'] = list(escolenc.inverse_transform(samp['ESCMAE']))

#### Overlap similarity

In [8]:
# Overlap

start = time.time() # start timer
similarity = sim.Overlap(samp[perfil_materno].to_numpy())
distmat_overlap = pd.DataFrame(similarity)
end = time.time() # end timer

print("time elapsed: ", (end - start)/60, " minutos")

time elapsed:  8.633101717631023  minutos


In [9]:
# save distance matrix
distmat_overlap.to_csv("overlap-distance-matrix.csv")

#path4 = "~/Documents/jupyter-workplace/data-mining-assignment/overlap-distance-matrix.csv"
#distmat_overlap = pd.read_csv(path4, index_col=0)